# Azure AI Agent Service နှင့် Semantic Kernel ကို အသုံးပြု၍ RAG

ဒီ code snippet က `Azure AI Agent Service` နဲ့ `Semantic Kernel` ကို အသုံးပြုပြီး retrieval-augmented generation (RAG) အတွက် Azure AI agent တစ်ခုကို ဖန်တီးပြီး စီမံခန့်ခွဲပုံကို ပြသထားပါတယ်။ Agent က အသုံးပြုသူရဲ့ မေးခွန်းတွေကို ရှာဖွေထားတဲ့ အကြောင်းအရာအခြေခံပြီး လိုက်လျောညီထွေဖြေကြားပေးပါတယ်။


SQLite ဗားရှင်းပြင်ဆင်မှု  
သင်အောက်ပါအမှားကိုတွေ့ရှိပါက -  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

သင်၏ notebook အစမှာ ဒီ code block ကို uncomment လုပ်ပါ:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### ပက်ကေ့ဂျ်များကို သွင်းယူခြင်း  
အောက်ပါကုဒ်သည် လိုအပ်သော ပက်ကေ့ဂျ်များကို သွင်းယူပါသည်။  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Retrieval-Augmented Generation with Semantic Kernel & Azure AI Agent Service

ဒီနမူနာက **Azure AI Agent Service** ကိုသုံးပြီး **Retrieval-Augmented Generation (RAG)** ကို ဘယ်လိုလုပ်ဆောင်ရမလဲဆိုတာကို ပြသပေးပါတယ်။ ဒါကို ဘာသာစကားမော်ဒယ်နဲ့ တစ်ဦးချင်းအထူးအချက်အလက်တွေ (တင်ထားတဲ့စာရွက်စာတမ်းထဲက) ကိုပေါင်းစပ်ပြီးလုပ်ဆောင်ပါတယ်။

### ဘယ်လိုအလုပ်လုပ်သလဲ

1. **စာရွက်စာတမ်းတင်သွင်းခြင်း**: Markdown ဖိုင်တစ်ခု (document.md) ကို agent service ထဲသို့တင်သွင်းသည်။ (ဥပမာ - Contoso ရဲ့ ခရီးသွားအာမခံမူဝါဒ)

2. **Vector Store ဖန်တီးခြင်း**: စာရွက်စာတမ်းကို vector store ထဲသို့ index လုပ်ပြီး၊ အကြောင်းအရာအပေါ် semantic search လုပ်နိုင်အောင် ပြင်ဆင်သည်။

3. **Agent ကို ပြင်ဆင်ခြင်း**: `gpt-4o` မော်ဒယ်ကို အသုံးပြုပြီး အောက်ပါအညွှန်းများဖြင့် agent ကို ဖန်တီးသည်။
   - စာရွက်စာတမ်းထဲက ရှာဖွေထားသော အချက်အလက်အပေါ်မူတည်ပြီးသာ မေးခွန်းများကို ဖြေဆိုရမည်။
   - မေးခွန်းသည် စာရွက်စာတမ်း၏ အကျယ်အဝန်းအပြင်ဖြစ်ပါက ဖြေဆိုရန်ငြင်းဆိုရမည်။

4. **File Search Tool ပေါင်းစည်းခြင်း**: `FileSearchTool` ကို agent နှင့် ပေါင်းစည်းပြီး၊ မော်ဒယ်သည် စာရွက်စာတမ်းထဲမှ index လုပ်ထားသော အချက်အလက်များကို ရှာဖွေပြီး အဓိကအချက်အလက်များကို ပြန်လည်ရယူနိုင်စေသည်။

5. **အသုံးပြုသူနှင့် အပြန်အလှန်ဆက်သွယ်မှု**: အသုံးပြုသူများသည် မေးခွန်းများမေးနိုင်သည်။ စာရွက်စာတမ်းထဲတွင် သက်ဆိုင်ရာအချက်အလက်များရှိပါက agent သည် အခြေခံထားသော အဖြေကို ဖန်တီးပေးသည်။  
   သက်ဆိုင်ရာအချက်အလက်မတွေ့ပါက စာရွက်စာတမ်းတွင် လုံလောက်သောအချက်အလက်မရှိကြောင်း agent သည် တိတိကျကျ ပြန်လည်ဖြေဆိုပေးသည်။


### အဓိက Function



Azure CLI ကို အသုံးပြု၍ `az login` ကို အရင်ဆုံး run လုပ်ပါ၊ ဒါမှ `DefaultAzureCredential` ကို အသုံးပြုတဲ့အခါ မှန်ကန်တဲ့ authentication context ကို ပေးနိုင်ပါမယ်။ Azure AI Agent Service က API keys မသုံးပါ။


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

သင့် Markdown ဖိုင်ကို ဘာသာပြန်ထားသည်မှာ အောက်ပါအတိုင်းဖြစ်ပါသည်။

# အကြောင်းအရာ

ဒီဖိုင်မှာ Markdown ဖိုင်ကို ဘာသာပြန်ပေးထားပြီး၊ အရေးကြီးသော စည်းကမ်းများကို လိုက်နာထားပါသည်။

## အရေးကြီးသော စည်းကမ်းများ

1. '''markdown''' သို့မဟုတ် အခြားသော tag များကို ဘာသာပြန်မှုတွင် မထည့်ပါနှင့်။
2. ဘာသာပြန်မှုသည် အလွန် တိကျသော အနက်မရောက်အောင် သေချာပါ။
3. မှတ်ချက်များကိုလည်း ဘာသာပြန်ပါ။
4. ဒီဖိုင်သည် Markdown format ဖြင့် ရေးထားသည် - XML သို့မဟုတ် HTML အဖြစ် မဆက်ဆံပါနှင့်။
5. အောက်ပါအရာများကို မဘာသာပြန်ပါနှင့်။
   - [!NOTE], [!WARNING], [!TIP], [!IMPORTANT], [!CAUTION]
   - Variable names, function names, class names
   - @@INLINE_CODE_x@@ သို့မဟုတ် @@CODE_BLOCK_x@@ ကဲ့သို့သော Placeholder များ
   - URLs သို့မဟုတ် paths
6. မူရင်း Markdown format အားလုံးကို အတိအကျ ထိန်းသိမ်းထားပါ။
7. ဘာသာပြန်ထားသော အကြောင်းအရာကိုသာ ပြန်ပေးပါ - အပို tag သို့မဟုတ် markup မထည့်ပါနှင့်။

## ဥပမာ

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရ အရင်းအမြစ်အဖြစ် ရှုလေ့လာသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားယူမှားမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
